In [2]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
print("Inicio da configuração inicial para a Oficina")

In [ ]:
print("Confere a configuração do boot.py")

In [ ]:
f = open("boot.py")
print(f.read())
f.close()

In [ ]:
print("Gerar arquivo de configuração para Webrepl")

In [ ]:
%sendtofile webrepl_cfg.py
PASS = '123456'

In [ ]:
print("Gerar arquivo de configuração conexão WIFI")

In [ ]:
%sendtofile net_cfg.py
SSID = 'MESQUITA2'
PASSWD = 'latinoware'

In [ ]:
print("Gerar novo boot.py")

In [ ]:
%sendtofile boot.py
# This file is executed on every boot (including wake-boot from deepsleep)
import gc
import net_cfg as net
def do_connect(SSID,PASSWD):
    import network
    sta_if = network.WLAN(network.STA_IF)
    if not sta_if.isconnected():
        print('connecting to network...')
        sta_if.active(True)
        sta_if.connect(SSID,PASSWD)
        while not sta_if.isconnected():
            pass
    print('network config:', sta_if.ifconfig())
    net_ip   = sta_if.ifconfig()[0]
    net_mask = sta_if.ifconfig()[1]
    net_router = sta_if.ifconfig()[2]
    net_dns = sta_if.ifconfig()[3]
    sta_if.ifconfig((net_ip, net_mask, net_router, '8.8.8.8'))
    import machine
    import time
    D1 = machine.Pin(5,machine.Pin.OUT)
    D2 = machine.Pin(4,machine.Pin.OUT)
    from machine import Pin, I2C
    import ssd1306
    i2c = I2C(sda=D2, scl=D1)
    display = ssd1306.SSD1306_I2C(128, 64, i2c)
    display.poweron()
    display.text(sta_if.ifconfig()[0], 0, 0, 1)
    display.show()

do_connect(net.SSID,net.PASSWD)
import webrepl
webrepl.start()
gc.collect()

In [ ]:
import webrepl
webrepl.start()

In [ ]:
import network
sta_if = network.WLAN(network.STA_IF)
sta_if.isconnected()
print(sta_if.ifconfig())

In [ ]:
%rebootdevice

In [ ]:
%disconnect

In [ ]:
%serialconnect to --port=/dev/ttyUSB2 --baud=115200

In [3]:
import network
sta_if = network.WLAN(network.STA_IF)
sta_if.isconnected()
print(sta_if.ifconfig())

('192.168.253.149', '255.255.255.0', '192.168.253.46', '8.8.8.8')


In [4]:
%disconnect

In [ ]:
import sys
sys.path.reverse()

In [ ]:
import upip
upip.install("micropython-umqtt.simple2")

In [ ]:
%disconnect

In [ ]:
%websocketconnect --password 123456 ws://192.168.2.161:8266/

In [ ]:
import umqtt


In [ ]:
from umqtt.simple import MQTTClient


In [ ]:
mq = MQTTClient("esp8266","climaduino.monitoracerrado.org",8883,"","")
#o connect deve retornar 0
mq.connect()
mq.publish(b"topico/luz_da_bancada",b"1")

In [ ]:
def do_connect(SSID,PASSWD):
    import network
    sta_if = network.WLAN(network.STA_IF)
    if not sta_if.isconnected():
        print('connecting to network...')
        sta_if.active(True)
        sta_if.connect(SSID,PASSWD)
        while not sta_if.isconnected():
            pass
    print('network config:', sta_if.ifconfig())

do_connect(net.SSID,net.PASSWD)

In [5]:
import urequests
url="https://onesait.community.o2br.com/api-manager/server/api/v1/EnergyAPI/65267f4596e3c7523c50f904"
response = urequests.get(url , headers={'accept': 'application/json','X-OP-APIKey':'2a083fa1f64749d39dc6d1e7f5eaa656'})
print(response.content)


b'[ {"_id": {"$oid": "65267f4596e3c7523c50f904"}, "Sensor": "tasmota_9E2FB3", "Dados": {"Time": "2023-10-11T11:56:01", "ENERGY": {"TotalStartTime": "2023-01-16T02:56:56", "Total": 66.85, "Yesterday": 0.229, "Today": 0.286, "Period": 0, "Power": 27, "ApparentPower": 32, "ReactivePower": 17, "Factor": 0.85, "Voltage": 218, "Current": 0.145}}}]'


In [ ]:
import os

In [ ]:
print(os.listdir())

In [ ]:
import webrepl
webrepl.start()

In [ ]:
%websocketconnect --password 123456 ws://192.168.2.103:8266/

In [ ]:
import os

In [ ]:
print(os.listdir("lib"))

In [ ]:
from umqtt.simple import MQTTClient

# Test reception e.g. with:
# mosquitto_sub -t foo_topic
c = MQTTClient("umqtt_client", "iothub02.onesaitplatform.o2br.com",8883,"","")
c.connect()
c.publish(b"foo_topic", b"hello")
c.disconnect()


In [ ]:
import time
from umqtt.robust import MQTTClient


def sub_cb(topic, msg):
    print((topic, msg))


c = MQTTClient("umqtt_client", "iothub02.onesaitplatform.o2br.com")
# Print diagnostic messages when retries/reconnects happens
c.DEBUG = True
c.set_callback(sub_cb)
if not c.connect(clean_session=False):
    print("New session being set up")
    c.subscribe(b"foo_topic")

while 1:
    c.wait_msg()

c.disconnect()
